In [1]:
import numpy
from sklearn.metrics import accuracy_score
import tensorflow as tf
import matplotlib.pyplot as plt

from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.shapelets import LearningShapelets, \
    grabocka_params_to_shapelet_size_dict
from tslearn.utils import ts_size
import numpy
from sklearn.metrics import accuracy_score

from tslearn.generators import random_walk_blobs
from tslearn.preprocessing import TimeSeriesScalerMinMax, \
    TimeSeriesScalerMeanVariance
from tslearn.neighbors import KNeighborsTimeSeriesClassifier, \
    KNeighborsTimeSeries
import pandas as pd
import numpy as np
import random
from tslearn.datasets import UCR_UEA_datasets
import sktime
from tslearn.preprocessing import TimeSeriesResampler
import random
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_unit_test
from sktime.classification.feature_based import Catch22Classifier
from sktime.datasets import load_unit_test
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_pickle("Train.pkl")
randomlist1 = random.sample(range(0, 260), 78)

df_test1 = df.loc[df["Batch"].isin(randomlist1)]
df_train1 = df.loc[~df["Batch"].isin(randomlist1)]
df_steps1 = df_test1.loc[df_test1["CuStepNo ValueY"] != df_test1["Next_Step"]]
df_steps2 = df_train1.loc[df_train1["CuStepNo ValueY"] != df_train1["Next_Step"]]

In [3]:
y_train1 = df_steps2.loc[df_steps2["CuStepNo ValueY"] == 1,"DeviationID ValueY"].astype(int)
y_test1 = df_steps1.loc[df_steps1["CuStepNo ValueY"] == 1,"DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)

In [4]:
X_train_step1 = []
for i in X_train1.Batch.unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == 1)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_train_step1.append(num)
X_test_step1 = []
for i in X_test1.Batch.unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == 1)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_test_step1.append(num)

In [5]:
length = []
for i in X_train_step1:
    length.append(len(i))

In [6]:
X_train_s1 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step1)
X_test_s1 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_test_step1)

In [7]:
X_train = X_train_s1.reshape(X_train_s1.shape[0],39,X_train_s1.shape[1])
X_test = X_test_s1.reshape(X_test_s1.shape[0],39,X_test_s1.shape[1])

In [8]:
clf = Catch22Classifier(
    estimator=RandomForestClassifier(n_estimators=5),
    outlier_norm=True,
)
clf.fit(X_train, y_train1)


Catch22Classifier(estimator=RandomForestClassifier(n_estimators=5),
                  outlier_norm=True)

In [9]:
y_pred = clf.predict(X_test)
accuracy_score(y_test1, y_pred)

0.76

In [42]:
g = clf.get_fitted_params()

In [45]:
d = g["classifier__feature_importances"]

In [48]:
d

array([0.        , 0.        , 0.        , 0.02606701, 0.        ,
       0.        , 0.01567045, 0.        , 0.00826488, 0.        ,
       0.0050504 , 0.00067092, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.02001251, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00219572, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.00116565, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00422336, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01404265, 0.        , 0.        , 0.        , 0.        ,
       0.02373689, 0.        , 0.        , 0.        , 0.     

In [26]:
clf.get_param_names()

['estimator', 'n_jobs', 'outlier_norm', 'random_state', 'replace_nans']

In [35]:
clf._check_classifier_input(X_test)

{'is_empty': False,
 'is_univariate': False,
 'is_equally_spaced': True,
 'is_equal_length': True,
 'n_instances': 75,
 'is_one_series': False,
 'n_panels': 1,
 'is_one_panel': True,
 'has_nans': False,
 'mtype': 'numpy3D',
 'scitype': 'Panel'}

In [40]:
clf

Catch22Classifier(estimator=RandomForestClassifier(n_estimators=5),
                  outlier_norm=True)